# Twitter Scrape

Traversing a whole lot of shit...

In [1]:
# Libraries
import pandas as pd
import numpy as np
import sys, os
from time import time

from sklearn.externals import joblib

from multiprocessing import Pool

In [2]:
# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

# Not Helpful, keeping for (maybe) future
#from ldig.predict_tweet import *
#detector = Detector('../src/ldig/models/model.latin/')

In [3]:
# Directories
ext_dir = '../data/external/'
proc_dir = '../data/processed/'

scrape_in = ext_dir + 'scrape/'
scrape_out = proc_dir + 'scrape/3-31/{}-extract.csv'

In [4]:
# cols to keep
cols_scrape = [
    'username',
    'longitude',
    'latitude',
    'date',
    'message',
    'tweetID',
    'language',
    'hashtag1',
    'hashtag2',
    'hashtag3',
    'hashtag4',
    'hashtag5'
]
cols_out = [
    'username',
    'longitude',
    'latitude',
    'date',
    'message',
    'hashtag1',
    'hashtag2',
    'hashtag3',
    'hashtag4',
    'hashtag5'
]

In [5]:
# Classifier
clf = joblib.load('../data/processed/class/3-28/model.pkl')

# to prepare us for processing raw tweets
%run ../src/twitter-sentiment/preprocessing/__init__.py

In [6]:
# List of all the files we'll be evaluating

# testing
#scrape_f = os.listdir(scrape_in)

# 'production'
scrape_f = [x for x in  os.listdir(scrape_in)
           if (x.split('.')[0][-4:] != '_log')]

In [7]:
def process_cleaning(tweet):
    try:
        return process_one(tweet)
    except:
        #print (tweet)
        return None

def process_stemming(tweet):
    try:
        return stem_one(tweet)
    except:
        #print (tweet)
        return None
    
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, 10)
    pool = Pool(4)
    #df = pd.concat(pool.map(func, df_split))
    df = [pool.map(func, df_split)]
    pool.close()
    pool.join()
    return df

def parallelize_series(series, func):
    pool = Pool(6)

    df = pool.map(func, series)

    pool.close()
    pool.join()
    return df

In [8]:
def process_scrape_csv(csv_f, verbose=False):
    """
    """
    if (verbose):
        # Set a pattern for verbosity
        metrics = {'f_name': csv_f}
        start = time()
        print (csv_f)
    
    # read in
    f_ = csv_f
    df = pd.read_csv(f_, usecols=cols_scrape, dtype={'message': str})
    if (verbose):
        metrics['orig'] = df.shape[0]
        print ("Number of tweets\n\tOriginal:\t\t{}".format(metrics['orig']))
        
    # pre screen our messages
    df.dropna(subset=['message'], inplace=True)
    if (verbose):
        metrics['message'] = df.shape[0]
        print ("\tWith messages:\t\t{}".format(metrics['message']))
        
    # filter, keep only english tweets
    df = df[df.language == 'en']
    if (verbose):
        metrics['eng'] = df.shape[0]
        print ("\tLang (twit.) eng.:\t{}".format(metrics['eng']))
    
    # Predict language
    ## Tested and it doesn't help...
    #df['pred_lang'] = parallelize_series(df.message.values, detector.get_tweet_lang)
    #english = df[df.pred_lang == 'en']
    #if (verbose):
        #print ("\tLang (class.) eng.:\t\t\t{}".format(df.shape[0]))
    
    # tokenize
    df['clean'] = parallelize_series(df.message.values, process_cleaning)
    df = df[pd.notnull(df['clean'])]
    if (verbose):
        metrics['clean'] = df.shape[0]
        print ("\tWith viable tokens:\t{}".format(metrics['clean']))
        
    # stem
    df['stem'] = parallelize_series(df.clean.values, process_stemming)
    df = df[pd.notnull(df['stem'])]
    if (verbose):
        metrics['stem'] = df.shape[0]
        print ("\tWith stems:\t\t{}".format(metrics['stem']))
    
    # CLASSIFY
    df['label'] = clf.predict(df.stem.values)
    df = df[df.label == 1]
    if (verbose):
        metrics['label'] = df.shape[0]
        print ("\tClassified+:\t\t{}".format(metrics['label']))
        
    #if verbose, lets log that out
    if (verbose):
        secs = time() - start
        metrics['time'] = secs
        print ("... in {} seconds\n".format(secs))
        
        log.append(metrics)
    
    return df

In [9]:
# Keep a dataframe for a log
#log_df = pd.DataFrame(columns=['f_name', 'time', 'orig', 'message', 'eng', 'clean', 'stem', 'label'])
log = []

In [12]:
for f in scrape_f:
    
    # file names
    f_in = scrape_in + f
    f_out = scrape_out.format(f[:-4])
    f_lat = f_out[:-4] + '-gps.csv'
    
    # sanity check for times sake
    if (os.path.isfile(f_lat)):
        continue
    else:
        try:
            # read in new tweets
            labeled_df = process_scrape_csv(f_in, verbose=True)

            # write out the now filtered csv
            #labeled_df.to_csv(out_f, index=False)

            labeled_df[pd.notnull(labeled_df['longitude'])].to_csv(f_lat, columns=cols_out, index=False)
        
        except:
            log.append({'f_name': f, 'orig': None})
            print ('Could not read.\n')

../data/external/scrape/tweets_immigrant_34401.csv
../data/external/scrape/tweets_immigrant_34402.csv
Number of tweets
	Original:		767700
	With messages:		763524
	Lang (twit.) eng.:	487299
	With viable tokens:	487299
	With stems:		487298
	Classified+:		3681
... in 56.087602853775024 seconds

../data/external/scrape/tweets_immigrant_34403.csv
Number of tweets
	Original:		792000
	With messages:		788116
	Lang (twit.) eng.:	489843
	With viable tokens:	489843
	With stems:		489843
	Classified+:		3412
... in 53.09950494766235 seconds

../data/external/scrape/tweets_immigrant_34404.csv
Number of tweets
	Original:		759017
	With messages:		755611
	Lang (twit.) eng.:	428314
	With viable tokens:	428314
	With stems:		428314
	Classified+:		3410
... in 46.28861093521118 seconds

../data/external/scrape/tweets_immigrant_34405.csv
Number of tweets
	Original:		719100
	With messages:		716098
	Lang (twit.) eng.:	410867
	With viable tokens:	410867
	With stems:		410867
	Classified+:		3855
... in 45.27227687

/home/rlrson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (1,2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Number of tweets
	Original:		424801
	With messages:		423212
	Lang (twit.) eng.:	222906
	With viable tokens:	222906
	With stems:		222906
	Classified+:		1047
... in 23.87152099609375 seconds

../data/external/scrape/tweets_immigrant_34469.csv
Number of tweets
	Original:		541900
	With messages:		540683
	Lang (twit.) eng.:	234571
	With viable tokens:	234571
	With stems:		234570
	Classified+:		1528
... in 25.11314296722412 seconds

../data/external/scrape/tweets_immigrant_34470.csv
Number of tweets
	Original:		493500
	With messages:		492058
	Lang (twit.) eng.:	233248
	With viable tokens:	233248
	With stems:		233248
	Classified+:		1491
... in 24.051504850387573 seconds

../data/external/scrape/tweets_immigrant_34471.csv
Number of tweets
	Original:		480400
	With messages:		479128
	Lang (twit.) eng.:	226144
	With viable tokens:	226144
	With stems:		226144
	Classified+:		1254
... in 23.909499645233154 seconds

../data/external/scrape/tweets_immigrant_34472.csv
Number of tweets
	Original:		52400

In [13]:
log_df = pd.DataFrame.from_records(log)
log_df.to_csv('../data/processed/scrape/3-31/log-(part 2).csv', index=False)

In [14]:
log_1 = pd.read_csv('../data/processed/scrape/3-31/log-(part 1).csv')
log_2 = pd.read_csv('../data/processed/scrape/3-31/log-(part 2).csv')

In [17]:
log_test = pd.concat([log_1, log_2], ignore_index=True)
log_test.to_csv('../data/processed/scrape/3-31/log.csv')